# Ejercicio 1: 

### Usa la clase PairWiseAligner de Biopython para alinear secuencias de ADN y obtener la puntuación y el alineamiento óptimo en estos dos escenarios.

In [1]:
from Bio.Align import PairwiseAligner
from Bio.Seq import Seq
import random
import pandas as pd
from itertools import product

### a) Generar las secuencias aleatoriamente (de forma automática no manual). Las dos secuencias pueden ser de diferente longitud.

#### 1. Generación de secuencias aleatorias

In [2]:
def generate_random_dna_sequence(length):
    return ''.join(random.choice('ACGT') for _ in range(length))

In [3]:
seq1 = Seq(generate_random_dna_sequence(random.randint(10, 20)))
seq2 = Seq(generate_random_dna_sequence(random.randint(15, 25)))

print(seq1)
print(seq2)

ATTCGGGGGATT
CCGGGTCAGAGGTGTCCCGGATGT


#### 2. Alineamiento de secuencias mediante búsqueda en rejilla

In [8]:
param_grid = {
    "mode": ["global", "local"],
    "match_score": [1, 2, 3],
    "mismatch_score": [-1, -2, -3],
    "open_gap_score": [-2, -3, -4],
    "extend_gap_score": [-0.5, -1, -1.5],
}

In [9]:
grid_combinations = list(product(
    param_grid["mode"],
    param_grid["match_score"],
    param_grid["mismatch_score"],
    param_grid["open_gap_score"],
    param_grid["extend_gap_score"],
))

In [12]:
def align_with_parameters(seq1, seq2, **kwargs):
    aligner = PairwiseAligner()
    for key, value in kwargs.items():
        setattr(aligner, key, value)
    alignment = aligner.align(seq1, seq2)
    return alignment.score, alignment

In [15]:
results = []
for params in grid_combinations:
    param_dict = {
        "mode": params[0],
        "match_score": params[1],
        "mismatch_score": params[2],
        "open_gap_score": params[3],
        "extend_gap_score": params[4],
    }
    score, alignment = align_with_parameters(seq1, seq2, **param_dict)

    results.append({
        "params": param_dict,
        "score": score,
        "alignment": alignment,
    })
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="score", ascending=False)

In [18]:
results_global = results_df[results_df["params"].apply(lambda x: x["mode"] == "global")]
results_local = results_df[results_df["params"].apply(lambda x: x["mode"] == "local")]

In [22]:
best_global = results_global.iloc[0]
alignment_global = best_global["alignment"]

print("Best global alignment:")
print(best_global)

Best global alignment:
params       {'mode': 'global', 'match_score': 3, 'mismatch...
score                                                     14.5
alignment    <Bio.Align.PairwiseAlignments object at 0x0000...
Name: 54, dtype: object


### b)  Obtener las secuencias a partir de ficheros obtenidos de bases de datos  biológicas. Explicar cómo se obtuvieron.

### Prueba a cambiar los parámetros del alineador como el esquema de puntuación, las penalizaciones de alineamiento y la modalidad de alineamiento (global o local). Compara los resultados obtenidos con diferentes configuraciones y explica las diferencias.